## Exploratory Data Analysis


In [1]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis, shapiro, normaltest
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

sns.set(style="whitegrid")

# Ensure inline plots display in Jupyter Notebooks
%matplotlib inline